In [ ]:
%matplotlib inline

import os

import numpy as np
import pandas as pd
import geopandas as gpd

import scipy.integrate as integrate

from matplotlib import pyplot as plt

import seaborn as sns

In [ ]:
ari = np.array([10, 200, 500, 1000, 2000])
aep = 1./ari
labels = ari.astype(str)

In [ ]:
basepath = "C:/WorkSpace/swhaq/data/impact/2020"
events = ['007-09962-2a', '007-09962-2b', '007-09962-2c', '007-09962-2d']


In [ ]:
file1 = os.path.join(basepath, events[0], f"QFES_{events[0]}.shp")
df = gpd.read_file(file1)
df.set_index('SA1_MAIN16', inplace=True)

In [ ]:
# Assume loss for a 1-in10 year event is zero
df['RP10'] = 0.0
df['SLR10'] = 0.0
df['RP200'] = df.REPVAL*df.slr_mean
df['SLR200'] = df.slr_mean
df.drop('slr_mean', axis=1, inplace=True)

In [ ]:
file2 = os.path.join(basepath, events[1], f"QFES_{events[1]}.shp")
df2 = gpd.read_file(file2)
df2.set_index('SA1_MAIN16', inplace=True)
file3 = os.path.join(basepath, events[2], f"QFES_{events[2]}.shp")
df3 = gpd.read_file(file3)
df3.set_index('SA1_MAIN16', inplace=True)
file4 = os.path.join(basepath, events[3], f"QFES_{events[3]}.shp")
df4 = gpd.read_file(file4)
df4.set_index('SA1_MAIN16', inplace=True)


In [ ]:
df = df.join(df2['slr_mean'])
df['RP500'] = df.REPVAL*df.slr_mean
df['SLR500'] = df.slr_mean
df.drop('slr_mean', axis=1, inplace=True)

In [ ]:
df = df.join(df3['slr_mean'])
df['RP1000'] = df.REPVAL*df.slr_mean
df['SLR1000'] = df.slr_mean
df.drop('slr_mean', axis=1, inplace=True)

In [ ]:
df = df.join(df4['slr_mean'])
df['RP2000'] = df.REPVAL*df.slr_mean
df['SLR2000'] = df.slr_mean
df.drop('slr_mean', axis=1, inplace=True)

In [ ]:
df[['RP10', 'RP200', 'RP500', 'RP1000', 'RP2000']]

In [ ]:
df[['SLR10', 'SLR200', 'SLR500', 'SLR1000', 'SLR2000']]

In [ ]:
def func(x, y):
    return integrate.simps(np.sort(y), x)

In [ ]:
df['AAL'] = df.apply(lambda x: func(np.flip(aep), np.flip(x[['RP10', 'RP200', 'RP500', 'RP1000', 'RP2000']].values)), axis=1)
df['AAD'] = df.apply(lambda x: func(np.flip(aep), np.flip(x[['SLR10', 'SLR200', 'SLR500', 'SLR1000', 'SLR2000']].values)), axis=1)

In [ ]:
df['AAL'] = df.apply(lambda x: func(aep, x[['RP10', 'RP200', 'RP500', 'RP1000', 'RP2000']].values),axis=1)
df['AAD'] = df.apply(lambda x: func(aep, x[['SLR10', 'SLR200', 'SLR500', 'SLR1000', 'SLR2000']].values),axis=1)

In [ ]:
df.loc[df['AAD'].idxmax()]

In [ ]:
sns.displot(df['AAD'])

In [ ]:
df.to_file(os.path.join(basepath, "007-09962-AAL.shp"))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))

df.plot(column='AAL', scheme='percentiles', ax=ax, legend=True)

In [ ]:
x = np.sort(df.loc['31802147505'][['RP10', 'RP200', 'RP500', 'RP1000', 'RP2000']].values)
plt.semilogy(x, aep)
plt.xlabel(f"Loss [million $]")
plt.ylabel("Exceedance probability")
None

In [ ]:
x = df.loc['31802147505'][['SLR10', 'SLR200', 'SLR500', 'SLR1000', 'SLR2000']].values
plt.semilogy(np.sort(x),aep)
plt.xlabel(f"Loss ratio")
plt.ylabel("Exceedance probability")
None